In [ ]:
!pip install datasets transformers evaluate pandas numpy rouge_score peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=475585d532188442cb4970212bc68d0283bf455d75dcd8f2743a9c1b0e814b09
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a53

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [ ]:
from datasets import load_dataset

dataset = load_dataset("daily_dialog")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.remove_columns(["act", "emotion"])

In [ ]:
#start_prompt1 = f"The following conversation is happening between #Person1# and #Person2#.Predict the next appropriate response of the #Person2#.The response should always be polite. Do not use any vulgur words.\n\n"
#start_prompt2 = f"The following conversation is happening between #Person1# and #Person2#.Predict the next appropriate response of the #Person1#.The response should always be polite. Do not use any vulgur words.\n\n"

In [ ]:
start_prompt1 = f"The following text is a conversation between #Person1# and #Person2#.The text ended with #Person1#.#Person2# should speak now to continue or end the conversation.Predict the next appropriate response of the #Person2#.The response should always be polite.\n\n"
start_prompt2 = f"The following text is a conversation between #Person1# and #Person2#.The text ended with #Person2#.#Person1# should speak now to continue or end the conversation.Predict the next appropriate response of the #Person1#.The response should always be polite.\n\n"

In [ ]:
#start_prompt1 = f"The following text is a conversation between #Person1# and #Person2#.The conversation stopped after the #Person1#'s reply.#Person2# should speak now to continue or end the conversation.Assume that you are the #Person2# and predict the suitable reply to #Person1#.\n\n"
#start_prompt2 = f"The following text is a conversation between #Person1# and #Person2#.The conversation stopped after the #Person2#'s reply.#Person1# should speak now to continue or end the conversation.Assume that you are the #Person1# and predict the suitable reply to #Person2#.\n\n"

In [ ]:
start_prompt1

'The following text is a conversation between #Person1# and #Person2#.The text ended with #Person1#.#Person2# should speak now to continue or end the conversation.Predict the next appropriate response of the #Person2#.The response should always be polite.\n\n'

In [ ]:
endprompt1 = "\n\n#Person1#:"
endprompt2 = "\n\n#Person2#:"

In [ ]:
Dataset = dataset

In [ ]:
def prepare_data(Dataset):
    inputs = []
    outputs = []

    for i in range(len(Dataset)):
      dialogues = Dataset[i]['dialog']

      ind = 0
      Inputs = ""
      flag = 0

      for i in dialogues:
        if flag == 0:
          Inputs += "#Person1#:" + i + "\n"
          inputs.append(start_prompt1 + Inputs + endprompt2)
          if ind == len(dialogues) - 2:
            outputs.append(dialogues[ind+1])
            break
          outputs.append(dialogues[ind+1])
          ind = ind + 1
          flag = 1
        else:
          Inputs += "#Person2#:" + i + "\n"
          inputs.append(start_prompt2 + Inputs + endprompt1)
          if ind == len(dialogues) - 2:
            outputs.append(dialogues[ind+1])
            break
          outputs.append(dialogues[ind+1])
          ind = ind + 1
          flag = 0

    df = pd.DataFrame({'inputs': inputs, 'outputs': outputs})
    return df

In [ ]:
df_train = prepare_data(dataset['train'])


In [ ]:
df_train

,inputs,outputs
0,The following text is a conversation between #...,You know that is tempting but is really not g...
1,The following text is a conversation between #...,What do you mean ? It will help us to relax .
2,The following text is a conversation between #...,Do you really think so ? I don't . It will ju...
3,The following text is a conversation between #...,I guess you are right.But what shall we do ? ...
4,The following text is a conversation between #...,I suggest a walk over to the gym where we can...
...,...,...
76047,The following text is a conversation between #...,"Sure , this will be a really good picture ."
76048,The following text is a conversation between #...,I want a pair of locus .
76049,The following text is a conversation between #...,"Take a look at the ones on display , please ."
76050,The following text is a conversation between #...,I need size 41 .


In [ ]:
df_val = prepare_data(dataset['validation'])
df_test = prepare_data(dataset['test'])

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict


tdf = df_train
vdf = df_val
testdf = df_test
tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)
testdf = Dataset.from_pandas(testdf)


ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = testdf
print(ds)

DatasetDict({
    train: Dataset({
        features: ['inputs', 'outputs'],
        num_rows: 76052
    })
    validation: Dataset({
        features: ['inputs', 'outputs'],
        num_rows: 7069
    })
    test: Dataset({
        features: ['inputs', 'outputs'],
        num_rows: 6740
    })
})


In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    example['input_ids'] = tokenizer(example['inputs'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["outputs"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/76052 [00:00<?, ? examples/s]

Map:   0%|          | 0/7069 [00:00<?, ? examples/s]

Map:   0%|          | 0/6740 [00:00<?, ? examples/s]

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['inputs', 'outputs'])

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 76052
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 7069
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6740
    })
})

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
dialogues = tokenized_datasets['test']['input_ids']
human_baseline_summaries = tokenized_datasets['test']['labels']

In [ ]:
original_model_summaries = []

for prompt in df_test['inputs'][0:30]:

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)



In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=df_test['outputs'][0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
original_model_results

{'rouge1': 0.14218572005798613,
 'rouge2': 0.009636222910216718,
 'rougeL': 0.13557248447952602,
 'rougeLsum': 0.13658876102610884}

In [ ]:
peft_results = []

In [ ]:
alpha_lora = 38

In [ ]:
rank = 8

In [ ]:
dropout = 0.03

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=rank, # Rank
    lora_alpha=alpha_lora,
    target_modules=["q", "v"],
    lora_dropout=dropout,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
print(lora_config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>, inference_mode=False, r=8, target_modules={'v', 'q'}, lora_alpha=38, lora_dropout=0.03, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})


In [ ]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 884736
all model parameters: 248462592
percentage of trainable model parameters: 0.36%


In [ ]:
train_epochs = 10

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{number}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.

    num_train_epochs=train_epochs,
    logging_strategy = 'steps', # Fix the typo here
    logging_steps=train_epochs,
    max_steps=train_epochs
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"]
)

In [ ]:
peft_trainer.train()

peft_model_path=f"./peft-dialogue-summary-checkpoint-local{number}"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,36.137500


('./peft-dialogue-summary-checkpoint-local1/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local1/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local1/spiece.model',
 './peft-dialogue-summary-checkpoint-local1/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local1/tokenizer.json')

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       f'/content/peft-dialogue-summary-checkpoint-local{number}',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [ ]:
peft_model_outputs = []

for prompt in df_test['inputs'][0:30]:

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    peft_model_result = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_result[0], skip_special_tokens=True)
    peft_model_outputs.append(peft_model_text_output)

In [ ]:
peft_model_results = rouge.compute(
    predictions=peft_model_outputs,
    references=df_test['outputs'][0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
peft_model_results

{'rouge1': 0.16425343252841268,
 'rouge2': 0.017820477118722734,
 'rougeL': 0.15600135160590228,
 'rougeLsum': 0.15790519583534843}

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.5f}%')


Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 2.20677%
rouge2: 0.81843%
rougeL: 2.04289%
rougeLsum: 2.13164%


In [ ]:
peft_model_results["Improvement: rogue1"] = improvement[0]
peft_model_results["Improvement: rogue2"] = improvement[1]
peft_model_results["Improvement: rougeL"] = improvement[2]
peft_model_results["Improvement: rougeLsum"] = improvement[3]

peft_model_results["r"] = lora_config.r
peft_model_results["lora_alpha"] = lora_config.lora_alpha
peft_model_results["lora_dropout"] = lora_config.lora_dropout
peft_model_results['epochs'] = train_epochs
peft_model_results['Learning_rate'] = "4e-4"



In [ ]:
peft_results.append(peft_model_results)

In [ ]:
df = pd.DataFrame(peft_results)
df


,rouge1,rouge2,rougeL,rougeLsum,Improvement: rogue1,Improvement: rogue2,Improvement: rougeL,Improvement: rougeLsum,r,lora_alpha,lora_dropout,epochs,Learning_rate
0,0.164253,0.01782,0.156001,0.157905,0.022068,0.008184,0.020429,0.021316,8,38,0.03,10,4e-4
